# Imports

In [1]:
import tweepy
import numpy as np
import pandas as pd
import time
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
consumer_token = "FSdktl3u8Q4xhmEHeC7tt4Q6P"
consumer_secret = "pErt27djL4HCcBt0z9PU6w7zVFygfVsEP1Aqx2gQ70uYibqP7p"
access_key = "1902120404-I1QEwvP7uLEBRJefMMPdPSf4uChDnsNvzO8hFcj"
access_secret = "RZAp4AlaXoszBXqp8gkMJM1ed6TJzTBFCDS99248hV9i4"

In [3]:
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_key, access_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
def makeCleanRep(origSource, removeLink=True):
    """
    Returns a list of the sid, text, and date of a given source.
    """
    
    sid = origSource._json['id']
    text = origSource._json['full_text']
    date = origSource._json['created_at']
    
    if removeLink==True:
        nonLinks = []
        for word in text.split(" "):
            if  "http" not in word:
                nonLinks.append(word)
        text = " ".join(nonLinks)
            
    
    return [sid, text, date]

In [6]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except (tweepy.RateLimitError, tweepy.TweepError):
            print("Rate limit exceeded, I'm passing out for a bit peace")
            time.sleep(15 * 60)

# Collect Data

In [14]:
allData = []
ids = []
texts = []
dates = []

c = 0
for handle in ["technology", "business", "WSJbusiness", "BBCWorldBiz", "ForbesTech", "CNBCtech"]:
# for handle in ["technology"]:

    print("Starting", handle,  "...")
    hc = 0
#     cur = limit_handled(tweepy.Cursor(api.user_timeline, screen_name = "technology", tweet_mode='extended').items(5000))
    cur = tweepy.Cursor(api.user_timeline, screen_name = handle, tweet_mode='extended').items(3000)
    for source in limit_handled(cur):
#     for source in cur:
        clean = makeCleanRep(source)
    #     lowerTweet = clean[1].lower()
    #     if bname in lowerTweet:
    #         relevantTweetData.append(clean)
    #         print(clean)
        if clean[0] not in ids:
#             allData.append(clean)
            ids.append(clean[0])
            texts.append(clean[1])
            dates.append(clean[2])
            c += 1
            hc += 1
#             print(c, clean[0])


    #     if len(relevantTweetData) >= 10:
    #         break
    print("Gathered", hc, "tweets for", handle)

Starting technology ...
Rate limit exceeded, I'm passing out for a bit peace


KeyboardInterrupt: 

In [15]:
df = pd.DataFrame({"IDs":ids, "Texts":texts, "Dates":dates}, columns=["IDs", "Texts", "Dates"])
df

,IDs,Texts,Dates


In [9]:
postDf = df.copy()
sia = SIA()

postDf['Sentiment'] = pd.Series([sia.polarity_scores(text)['compound'] for text in postDf['Texts']])
postDf

,IDs,Texts,Dates,Sentiment
0,1022949841067016194,Don't miss it! #BTECH is streaming LIVE on @Tw...,Fri Jul 27 21:00:22 +0000 2018,0.1867
1,1022940008553041921,Internal combustion engines may have 40 more y...,Fri Jul 27 20:21:18 +0000 2018,0.0000
2,1022928924899196928,Facebook says EU privacy law caused a drop in ...,Fri Jul 27 19:37:15 +0000 2018,-0.5719
3,1022927338864963584,Amazon building a snowball effect with momentu...,Fri Jul 27 19:30:57 +0000 2018,0.1779
4,1022912356676976642,NYC City Council to consider a pause in granti...,Fri Jul 27 18:31:25 +0000 2018,0.3182
5,1022904377584087040,"Scandals have damaged Facebook, but the most p...",Fri Jul 27 17:59:42 +0000 2018,-0.7916
6,1022899235749064704,Twitter falls the most in 2 years on lower mon...,Fri Jul 27 17:39:17 +0000 2018,0.1280
7,1022887617782796289,Wall Street races to NXP's defense after the f...,Fri Jul 27 16:53:07 +0000 2018,-0.4215
8,1022881624726360065,Twitter's post-earnings rout shatters two key ...,Fri Jul 27 16:29:18 +0000 2018,0.0000
9,1022869269477031937,"In the age of cord cutting, many cable loyalis...",Fri Jul 27 15:40:12 +0000 2018,0.1531


In [10]:
postDf.to_csv("historical.csv",  index=False)

In [17]:
loadedDf = pd.read_csv("historical.csv")
loadedDf['Dates']

0        Fri Jul 27 21:00:22 +0000 2018
1        Fri Jul 27 20:21:18 +0000 2018
2        Fri Jul 27 19:37:15 +0000 2018
3        Fri Jul 27 19:30:57 +0000 2018
4        Fri Jul 27 18:31:25 +0000 2018
5        Fri Jul 27 17:59:42 +0000 2018
6        Fri Jul 27 17:39:17 +0000 2018
7        Fri Jul 27 16:53:07 +0000 2018
8        Fri Jul 27 16:29:18 +0000 2018
9        Fri Jul 27 15:40:12 +0000 2018
10       Fri Jul 27 15:05:07 +0000 2018
11       Fri Jul 27 14:37:25 +0000 2018
12       Fri Jul 27 14:23:50 +0000 2018
13       Fri Jul 27 12:33:19 +0000 2018
14       Fri Jul 27 12:10:32 +0000 2018
15       Fri Jul 27 11:54:16 +0000 2018
16       Fri Jul 27 11:45:04 +0000 2018
17       Fri Jul 27 11:23:35 +0000 2018
18       Fri Jul 27 11:20:17 +0000 2018
19       Fri Jul 27 10:59:53 +0000 2018
20       Fri Jul 27 10:59:13 +0000 2018
21       Fri Jul 27 10:39:20 +0000 2018
22       Fri Jul 27 10:16:45 +0000 2018
23       Fri Jul 27 09:54:53 +0000 2018
24       Fri Jul 27 09:31:45 +0000 2018
